In [2]:
import os
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [3]:
os.environ["OPENAI_API_KEY"] = ""

# load documents
loader = PyPDFLoader("Survey Neural Speech.pdf")
documents = loader.load()

# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# select which embeddings we want to use
embeddings = OpenAIEmbeddings()

# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)

# expose this index in a retrieval interface
retriever = db.as_retriever(
    search_type="similarity", search_kwargs={"k": 2}
)

# create a chain to answer questions
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="map_reduce",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

/Users/taiduong/.pyenv/versions/3.11.8/envs/project/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/taiduong/.pyenv/versions/3.11.8/envs/project/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  wa

In [4]:
qa("What does a moderm TTS system consist of?")

/Users/taiduong/.pyenv/versions/3.11.8/envs/project/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What does a moderm TTS system consist of?',
 'result': ' A modern TTS system usually consists of a text analysis module, a parameter prediction module (acoustic model), and a vocoder analysis/synthesis module (vocoder).',
 'source_documents': [Document(page_content='TTS SurveySec. 2: Key Components in TTSSec. 2.1: Main Taxonomy\nSec. 2.2: Text Analysis\nSec. 2.3: Acoustic Model\nSec. 2.4: V ocoder\nSec. 2.5: Towards Fully E2E TTS\nSec. 2.6: Other Taxonomies\nSec. 3: Advanced Topics in TTSSec. 3.1: Background and Taxonomy\nSec. 3.2: Fast TTS\nSec. 3.3: Low-Resource TTS\nSec. 3.4: Robust TTS\nSec. 3.5: Expressive TTS\nSec. 3.6: Adaptive TTS\nFigure 2: Organization of this survey paper.\nAdvanced Topics in TTS Besides the key components of neural TTS, we further review several\nadvanced topics in neural TTS, which push the frontier of TTS research and address practical\nchallenges in TTS product. For example, as TTS is a typical sequence to sequence generation task\nand the out

In [5]:
import panel as pn
pn.extension()

pdf_input = pn.widgets.FileInput(accept=".pdf", value="", height=50)
key_input = pn.widgets.PasswordInput(
    name="OpenAI Key",
    placeholder="sk-...",
)
k_slider = pn.widgets.IntSlider(
    name="Number of Relevant Chunks", start=1, end=5, step=1, value=2
)
chain_select = pn.widgets.RadioButtonGroup(
    name="Chain Type",
    options=["stuff", "map_reduce", "refine", "map_rerank"]
)
chat_input = pn.widgets.TextInput(placeholder="First, upload a PDF!")

In [6]:
def initialize_chain():
    if key_input.value:
        os.environ["OPENAI_API_KEY"] = key_input.value

    selections = (pdf_input.value, k_slider.value, chain_select.value)
    if selections in pn.state.cache:
        return pn.state.cache[selections]

    chat_input.placeholder = "Ask questions here!"

    # load document
    with tempfile.NamedTemporaryFile("wb", delete=False) as f:
        f.write(pdf_input.value)
    file_name = f.name
    loader = PyPDFLoader(file_name)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(
        search_type="similarity", search_kwargs={"k": k_slider.value}
    )
    # create a chain to answer questions
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(),
        chain_type=chain_select.value,
        retriever=retriever,
        return_source_documents=True,
        verbose=True,
    )
    return qa

In [7]:
async def respond(contents, user, chat_interface):
    if not pdf_input.value:
        chat_interface.send(
            {"user": "System", "value": "Please first upload a PDF!"}, respond=False
        )
        return
    elif chat_interface.active == 0:
        chat_interface.active = 1
        chat_interface.active_widget.placeholder = "Ask questions here!"
        yield {"user": "OpenAI", "value": "Let's chat about the PDF!"}
        return

    qa = initialize_chain()
    response = qa({"query": contents})
    answers = pn.Column(response["result"])
    answers.append(pn.layout.Divider())
    for doc in response["source_documents"][::-1]:
        answers.append(f"**Page {doc.metadata['page']}**:")
        answers.append(f"```\n{doc.page_content}\n```")
    yield {"user": "OpenAI", "value": answers}


chat_interface = pn.chat.ChatInterface(
    callback=respond, sizing_mode="stretch_width", widgets=[pdf_input, chat_input]
)
chat_interface.send(
    {"user": "System", "value": "Please first upload a PDF and click send!"},
    respond=False,
)

TypeError: standalone_docs_json_and_render_items() takes 1 positional argument but 2 were given

ChatMessage(str, avatar='⚙️', reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='System')

In [8]:
template = pn.template.BootstrapTemplate(
    sidebar=[key_input, k_slider, chain_select], main=[chat_interface]
)
template.servable()

BootstrapTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] BootstrapTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-4573763088] ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Tabs, _placeholder=ChatMessage, _widgets={'FileInput': FileInput(ac...}, callback=<function respond a..., show_button_name=True, sizing_mode='stretch_width', widgets=[FileInput(accept='.pdf', ...])
        [0] ChatMessage(str, avatar='⚙️', reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='System')
    [nav-6015840592] PasswordInput(name='OpenAI Key', placeholder='sk-...')
    [nav-6015841296] IntSlider(end=5, name='Number of Relevant C..., start=1, value=2)
    [nav-6016250320] RadioButtonGroup(name='Chain Type', options=['stuff', 'map_reduce', ...], value='stuff')